In [1]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import xmltodict
import json
import numpy as np
import gc
import os

In [2]:
dir_dict=dict()
root = './이상행동 CCTV 영상'
class_list = os.listdir(root)

for class_folder in class_list:
    print(class_folder)
    dir_dict[class_folder]=[]
    place_list = os.listdir(root + '/'+ class_folder)
    for place_num in place_list:
        print(place_num)
        num_list = os.listdir(root + '/'+ class_folder + '/'+ place_num)
        for num in num_list:
            #print(num)
            file_list = os.listdir(root + '/'+ class_folder + '/'+ place_num + '/'+ num)
            for file in file_list:
                #print(file)
                if(file[-4:]=='.mp4'):
                    dir_dict[class_folder].append(root + '/'+ class_folder + '/'+ place_num + '/'+ num + '/'+ file[:-4])
                #print(file[:-4])
                #print(file[-4:]=='.mp4')
            
        

01.폭행(assult)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
02.싸움(fight)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
03.절도(burglary)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
04.기물파손(vandalism)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
05.실신(swoon)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
06.배회(wander)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
07.침입(trespass)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
08.투기(dump)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
09.강도(robbery)
outsidedoor_01
outsidedoor_02
outsidedoor_03
outsidedoor_04
10.데이트폭력및추행(datefight)
outsidedoor_01
outsidedoor_02
outsidedoor_02.zip


NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: './이상행동 CCTV 영상/10.데이트폭력및추행(datefight)/outsidedoor_02.zip'

In [3]:
dir_dict

{'01.폭행(assult)': ['./이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-2/11-2_cam01_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-2/11-2_cam01_assault01_place08_night_summer',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-2/11-2_cam02_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-2/11-2_cam02_assault01_place08_night_summer',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-3/11-3_cam01_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-3/11-3_cam02_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-4/11-4_cam01_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-4/11-4_cam01_assault01_place08_night_summer',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-4/11-4_cam02_assault01_place08_night_spring',
  './이상행동 CCTV 영상/01.폭행(assult)/outsidedoor_01/11-4/11-4_cam02_assault01_place08_night_summer',
  './이상행동 CCTV 영상/01.폭행

In [4]:
class_dict=dict()
class_dict['normal']=0
class_dict['assault']=1
class_dict['fight']=2
class_dict['burglary']=3
class_dict['vandalism']=4
class_dict['swoon']=5
class_dict['wander']=6
class_dict['trespass']=7
class_dict['dump']=8
class_dict['robbery']=9
class_dict['datefight']=10
class_dict['kidnap']=11
class_dict['drunken']=12

In [5]:
def mk_mp4_label(file_path, out_path, num_seqence=16, fps_setting=5):
    xml_path = file_path + '.xml'
    video_file = file_path + '.mp4'
    num_seq = num_seqence
    fps_set = fps_setting
    resize_width = 172
    resize_height = 172
    
    #xml 파싱
    f = open(xml_path, 'r')
    read = f.read()
    dict2_type = xmltodict.parse(read)

    label=dict()
    key_pos=[0,0]
    
    if type(dict2_type['annotation']['object'])==type([]):
        for obj in range(len(dict2_type['annotation']['object'])):
            key_pos[0]+=int(dict2_type['annotation']['object'][obj]['position']['keypoint']['x'])
            key_pos[1]+=int(dict2_type['annotation']['object'][obj]['position']['keypoint']['y'])
        key_pos[0] = int(key_pos[0]/len(dict2_type['annotation']['object']))
        key_pos[1] = int(key_pos[1]/len(dict2_type['annotation']['object']))
    else:
        key_pos[0]+=int(dict2_type['annotation']['object']['position']['keypoint']['x'])
        key_pos[1]+=int(dict2_type['annotation']['object']['position']['keypoint']['y'])

    st = dict2_type['annotation']['event']['starttime'].split(':')
    du = dict2_type['annotation']['event']['duration'].split(':')

    if len(st)==3:
        stint = int(((float(st[0])*3600) + (float(st[1])*60) + float(st[2])) * int(dict2_type['annotation']['header']['fps']))
    else:
        stint = int(((float(st[0])*60) + float(st[1])) * int(dict2_type['annotation']['header']['fps']))
    if len(du)==3:
        etint = stint + int(((float(du[0])*3600) + (float(du[1])*60) + float(du[2])) * int(dict2_type['annotation']['header']['fps']))
    else:
        etint = stint + int(((float(du[0])*60) + float(du[1])) * int(dict2_type['annotation']['header']['fps']))


    #크롭 및 리사이즈            
    width = int(dict2_type['annotation']['size']['width'])
    height = int(dict2_type['annotation']['size']['height'])
    depth = int(dict2_type['annotation']['size']['depth'])

    if width>height:
        low=height
    else:
        low=width

    xs = key_pos[0]-int(low/2)
    xe = key_pos[0]+int(low/2)
    ys = key_pos[1]-int(low/2)
    ye = key_pos[1]+int(low/2)

    
    if xs < 0:
        xe = xe - xs
        xs = 0
    if ys < 0:
        ye = ye - ys
        ys=0
    if xe > width:
        xs = xs - (xe - width)
        xe = width
    if ye > height:
        ys = ys - (ye - height)
        ye = height


    start_end=[]
    for obj in label:
        #print(obj)
        for act in label[obj]:
            #print(act)
            for se in label[obj][act]:
                #print(se)
                start_end.append([act, se[0], se[1]])
                
    fps_dev = int(int(dict2_type['annotation']['header']['fps'])/fps_set)
    cap = cv2.VideoCapture(video_file)
    
    label=[]
    cla = class_dict[dict2_type['annotation']['event']['eventname']]
    i=0
    
    out = cv2.VideoWriter(filename = out_path + '.mp4',fourcc=cv2.VideoWriter_fourcc(*'DIVX'), 
                      fps = fps_setting, frameSize = (resize_width, resize_height), isColor = True)
    if not out.isOpened():
        print('out File open failed!', out_path + '.mp4')

    if cap.isOpened():
        while True:
            i+=1
            ret, img = cap.read()
            if ret:
                if i%fps_dev==0:
                    cropped_img = img[ys: ye, xs: xe]
                    resized_img = cv2.resize(cropped_img, dsize=(resize_width, resize_height), interpolation=cv2.INTER_AREA)
                    out.write(resized_img)
                    
                    if i > stint and i < etint:
                        label.append(cla)
                    else:
                        label.append(0)
                    
                if i%100==0:
                    gc.collect()
            else:
                break
        
        cap.release()
        out.release()
        
    else:
        print('cannot open the file', out_path + '.mp4')


    label2 = np.array(label, dtype=np.uint8)

    np.save(out_path + '.npy', label2)
    print('success', out_path)

In [6]:
for n, class_n in enumerate(dir_dict):
    if n==2:
        print(len(dir_dict[class_n]))
        video_num=0
        for path in dir_dict[class_n]:
            mk_mp4_label(file_path = path, out_path = './iterdata/'+str(n+1)+'/'+str(video_num), num_seqence=16, fps_setting=5)
            video_num += 1

217
success ./iterdata/3/0
success ./iterdata/3/1
success ./iterdata/3/2
success ./iterdata/3/3
success ./iterdata/3/4


KeyboardInterrupt: 